In [77]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import os
import datetime as dt

In [78]:
COLS = ['WordleID','Date','Total_Tweets','Total_NonTrunc_Tweets','Total_Extracted_Tweets','C1','P1','C2','P2','C3','P3','C4','P4','C5','P5','C6','P6','CX','PX','Total_Hard_Tweets',
       'PH1','PH2','PH3','PH4','PH5','PH6','PHX']

### Read The Dataset and analyse the data

In [79]:
df = pd.read_csv("../input/wordletweets/WordleMegaData.csv")
df.head()

,WordleID,ID,Created_At,Text,Source,UserID,Username,User_ScreenName,Location,Truncated
0,254,1498447921448034305,2022-02-28 23:59:58+00:00,Wordle 254 3/6\n\n🟨🟨⬛⬛⬛\n🟨⬛🟨⬛🟩\n🟩🟩🟩🟩🟩,Twitter for iPhone,36816757,Leslie Brown,live_laugh_pray,Haida Gwaii,False
1,254,1498447918184996864,2022-02-28 23:59:58+00:00,Wordle 254 4/6\n\n⬛⬛⬛⬛🟩\n⬛⬛⬛⬛🟩\n⬛🟩⬛⬛🟩\n🟩🟩🟩🟩🟩\n...,Twitter for iPhone,482591014,Dylan Sobo,DaBolt727,"Largo, FL / Bradenton|FGCU",False
2,254,1498447910173921282,2022-02-28 23:59:56+00:00,Wordle 254 3/6\n\n⬛⬛⬛⬛🟩\n⬛⬛🟩⬛🟩\n🟩🟩🟩🟩🟩,Twitter for iPhone,1397624255626584074,🖤,wengojos,23 • she/her • 🇵🇭🇺🇸,False
3,254,1498447901797801989,2022-02-28 23:59:54+00:00,Wordle 254 3/6\n\n⬛⬛⬛🟨⬛\n⬛🟨⬛⬛🟩\n🟩🟩🟩🟩🟩,Twitter for iPhone,1255905442548473856,Dustin Waters,DustinWaters12,NaN,False
4,254,1498447896911527938,2022-02-28 23:59:53+00:00,Wordle 254 3/6\n\n⬛🟨⬛⬛⬛\n🟩🟩🟩⬛🟩\n🟩🟩🟩🟩🟩,Twitter for iPhone,252893719,R. Colin,rollin_fatty,"Richmond, VA",False


### Size of The dataset (No of Rows)

In [80]:
len_orig = len(df)
print(f"Number of original rows = {len_orig}")

Number of original rows = 240000


### Count of all Wordle ID

In [81]:
wordleid_list = df.WordleID.unique()
wordleid_list

array([254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265])

In [82]:
len_wid_og = df.groupby("WordleID")["WordleID"].count()
len_wid_og

WordleID
254    20000
255    20000
256    20000
257    20000
258    20000
259    20000
260    20000
261    20000
262    20000
263    20000
264    20000
265    20000
Name: WordleID, dtype: int64

#### The data has truncated text which is marked with Column Truncated as true. Removing such data as full text is not available

In [83]:
df = df.drop(df[df.Truncated].index)
len_trunc = len(df)
print(f"Number of rows after removing truncated tweets = {len_trunc}")

Number of rows after removing truncated tweets = 235942


### Count of all Wordle ID after removing truncated rows

In [84]:
len_wid_ntrunc = df.groupby("WordleID")["WordleID"].count()
len_wid_ntrunc

WordleID
254    19727
255    19501
256    19703
257    19707
258    19658
259    19723
260    19750
261    19731
262    19687
263    19709
264    19756
265    19290
Name: WordleID, dtype: int64

In [85]:
data_acc = {}

In [86]:

for i in wordleid_list:
    data_acc[i] = {}
    data_acc[i]['WordleID'] = i
    data_acc[i]['Total_Tweets'] = len_wid_og[i]
    data_acc[i]['Total_NonTrunc_Tweets'] = len_wid_ntrunc[i]
    

In [87]:
data_acc

{254: {'WordleID': 254, 'Total_Tweets': 20000, 'Total_NonTrunc_Tweets': 19727},
 255: {'WordleID': 255, 'Total_Tweets': 20000, 'Total_NonTrunc_Tweets': 19501},
 256: {'WordleID': 256, 'Total_Tweets': 20000, 'Total_NonTrunc_Tweets': 19703},
 257: {'WordleID': 257, 'Total_Tweets': 20000, 'Total_NonTrunc_Tweets': 19707},
 258: {'WordleID': 258, 'Total_Tweets': 20000, 'Total_NonTrunc_Tweets': 19658},
 259: {'WordleID': 259, 'Total_Tweets': 20000, 'Total_NonTrunc_Tweets': 19723},
 260: {'WordleID': 260, 'Total_Tweets': 20000, 'Total_NonTrunc_Tweets': 19750},
 261: {'WordleID': 261, 'Total_Tweets': 20000, 'Total_NonTrunc_Tweets': 19731},
 262: {'WordleID': 262, 'Total_Tweets': 20000, 'Total_NonTrunc_Tweets': 19687},
 263: {'WordleID': 263, 'Total_Tweets': 20000, 'Total_NonTrunc_Tweets': 19709},
 264: {'WordleID': 264, 'Total_Tweets': 20000, 'Total_NonTrunc_Tweets': 19756},
 265: {'WordleID': 265, 'Total_Tweets': 20000, 'Total_NonTrunc_Tweets': 19290}}

### Attempts Count

In [88]:
attempt = df['Text'].str.lower().str.extract(r'([1-6Xx]/6)')
len_wordle_texts = np.sum(attempt.count())
print(f"Count of tweets with extracted text = {len_wordle_texts}")
print('-------------------')

attempt = attempt.groupby([0])[0].count().rename('Count').reset_index()
print('Count of user which got the wordle at which attempt: \n')
print(attempt)
print('-------------------')

# print(attempt['Count'].sum())

attempt['Percentage'] = ((attempt['Count']*100)/(attempt['Count'].sum())).round(2)
print('Count and Percentage of user which got the wordle at which attempt: \n')
print(attempt)
# print(x.loc[0][0],x.loc[0].Count)

Count of tweets with extracted text = 235114
-------------------
Count of user which got the wordle at which attempt: 

     0  Count
0  1/6   2907
1  2/6  18466
2  3/6  59851
3  4/6  72726
4  5/6  48729
5  6/6  25317
6  x/6   7118
-------------------
Count and Percentage of user which got the wordle at which attempt: 

     0  Count  Percentage
0  1/6   2907        1.24
1  2/6  18466        7.85
2  3/6  59851       25.46
3  4/6  72726       30.93
4  5/6  48729       20.73
5  6/6  25317       10.77
6  x/6   7118        3.03


### HardMode

In [89]:
hard_mode_attempt = df['Text'].str.lower().str.extract(r'([1-6Xx]/6\*)')
len_wordle_hard_texts = np.sum(hard_mode_attempt.count())
print(f"Count of hard mode tweets = {len_wordle_hard_texts}")
print('-------------------')

hard_mode_attempt = hard_mode_attempt.groupby([0])[0].count().rename('Count').reset_index()
print('Count of user which got the wordle at which attempt with hard mode: \n')
print(hard_mode_attempt)
print('-------------------')

hard_mode_attempt['Percentage'] = ((hard_mode_attempt['Count']*100)/(hard_mode_attempt['Count'].sum())).round(2)
print('Count and Percentage of user which got the wordle at which attempt: \n')
print(hard_mode_attempt)

Count of hard mode tweets = 8418
-------------------
Count of user which got the wordle at which attempt with hard mode: 

      0  Count
0  1/6*     62
1  2/6*    681
2  3/6*   2451
3  4/6*   2701
4  5/6*   1508
5  6/6*    687
6  x/6*    328
-------------------
Count and Percentage of user which got the wordle at which attempt: 

      0  Count  Percentage
0  1/6*     62        0.74
1  2/6*    681        8.09
2  3/6*   2451       29.12
3  4/6*   2701       32.09
4  5/6*   1508       17.91
5  6/6*    687        8.16
6  x/6*    328        3.90


In [90]:
def get_attempt_count(df):
    attempt = df['Text'].str.lower().str.extract(r'([1-6Xx]/6)')
    len_wordle_texts = np.sum(attempt.count())
#     print(f"Count of tweets with extracted text = {len_wordle_texts}")
#     print('-------------------')

    attempt = attempt.groupby([0])[0].count().rename('Count').reset_index()
#     print('Count of user which got the wordle at which attempt: \n')
#     print(attempt)
#     print('-------------------')

    # print(attempt['Count'].sum())

    attempt['Percentage'] = ((attempt['Count']*100)/(attempt['Count'].sum())).round(2)
#     print('Count and Percentage of user which got the wordle at which attempt: \n')
#     print(attempt)
    return attempt
    

In [91]:
def get_hardmode_count(df):
    hard_mode_attempt = df['Text'].str.lower().str.extract(r'([1-6Xx]/6\*)')
    len_wordle_hard_texts = np.sum(hard_mode_attempt.count())
#     print(f"Count of hard mode tweets = {len_wordle_hard_texts}")
#     print('-------------------')

    hard_mode_attempt = hard_mode_attempt.groupby([0])[0].count().rename('Count').reset_index()
#     print('Count of user which got the wordle at which attempt with hard mode: \n')
#     print(hard_mode_attempt)
#     print('-------------------')

    hard_mode_attempt['Percentage'] = ((hard_mode_attempt['Count']*100)/(hard_mode_attempt['Count'].sum())).round(2)
#     print('Count and Percentage of user which got the wordle at which attempt for hard mode: \n')
#     print(hard_mode_attempt)
    return hard_mode_attempt

In [92]:
for wid in wordleid_list:
    print(wid)
    x = df.loc[df['WordleID'] == wid]
    data_acc[wid]['Date'] = dt.datetime.strptime(x.iloc[0].Created_At,'%Y-%m-%d %H:%M:%S%z').strftime('%Y-%m-%d')
    attempt = get_attempt_count(x)
    data_acc[wid]['Total_Extracted_Tweets'] = attempt['Count'].sum()
    for i in range(len(attempt)) :
        data_acc[wid][f'C{i+1}'] = attempt.iloc[i, 1]
        data_acc[wid][f'P{i+1}'] = attempt.iloc[i, 2]
    
    hard_mode_attempt = get_hardmode_count(x)
    data_acc[wid]['Total_Hard_Tweets'] = hard_mode_attempt['Count'].sum()
    for i in range(len(hard_mode_attempt)) :
        data_acc[wid][f'PH{i+1}'] = hard_mode_attempt.iloc[i, 2]
    print(data_acc[wid])

254
{'WordleID': 254, 'Total_Tweets': 20000, 'Total_NonTrunc_Tweets': 19727, 'Date': '2022-02-28', 'Total_Extracted_Tweets': 19556, 'C1': 374, 'P1': 1.91, 'C2': 1895, 'P2': 9.69, 'C3': 5893, 'P3': 30.13, 'C4': 6610, 'P4': 33.8, 'C5': 3418, 'P5': 17.48, 'C6': 1214, 'P6': 6.21, 'C7': 152, 'P7': 0.78, 'Total_Hard_Tweets': 589, 'PH1': 0.68, 'PH2': 10.36, 'PH3': 35.48, 'PH4': 39.22, 'PH5': 11.21, 'PH6': 2.72, 'PH7': 0.34}
255
{'WordleID': 255, 'Total_Tweets': 20000, 'Total_NonTrunc_Tweets': 19501, 'Date': '2022-03-01', 'Total_Extracted_Tweets': 19429, 'C1': 200, 'P1': 1.03, 'C2': 599, 'P2': 3.08, 'C3': 3460, 'P3': 17.81, 'C4': 6534, 'P4': 33.63, 'C5': 5570, 'P5': 28.67, 'C6': 2668, 'P6': 13.73, 'C7': 398, 'P7': 2.05, 'Total_Hard_Tweets': 657, 'PH1': 0.76, 'PH2': 2.13, 'PH3': 22.68, 'PH4': 38.36, 'PH5': 27.85, 'PH6': 7.31, 'PH7': 0.91}
256
{'WordleID': 256, 'Total_Tweets': 20000, 'Total_NonTrunc_Tweets': 19703, 'Date': '2022-03-02', 'Total_Extracted_Tweets': 19641, 'C1': 345, 'P1': 1.76, 'C2

In [93]:
data_acc

{254: {'WordleID': 254,
  'Total_Tweets': 20000,
  'Total_NonTrunc_Tweets': 19727,
  'Date': '2022-02-28',
  'Total_Extracted_Tweets': 19556,
  'C1': 374,
  'P1': 1.91,
  'C2': 1895,
  'P2': 9.69,
  'C3': 5893,
  'P3': 30.13,
  'C4': 6610,
  'P4': 33.8,
  'C5': 3418,
  'P5': 17.48,
  'C6': 1214,
  'P6': 6.21,
  'C7': 152,
  'P7': 0.78,
  'Total_Hard_Tweets': 589,
  'PH1': 0.68,
  'PH2': 10.36,
  'PH3': 35.48,
  'PH4': 39.22,
  'PH5': 11.21,
  'PH6': 2.72,
  'PH7': 0.34},
 255: {'WordleID': 255,
  'Total_Tweets': 20000,
  'Total_NonTrunc_Tweets': 19501,
  'Date': '2022-03-01',
  'Total_Extracted_Tweets': 19429,
  'C1': 200,
  'P1': 1.03,
  'C2': 599,
  'P2': 3.08,
  'C3': 3460,
  'P3': 17.81,
  'C4': 6534,
  'P4': 33.63,
  'C5': 5570,
  'P5': 28.67,
  'C6': 2668,
  'P6': 13.73,
  'C7': 398,
  'P7': 2.05,
  'Total_Hard_Tweets': 657,
  'PH1': 0.76,
  'PH2': 2.13,
  'PH3': 22.68,
  'PH4': 38.36,
  'PH5': 27.85,
  'PH6': 7.31,
  'PH7': 0.91},
 256: {'WordleID': 256,
  'Total_Tweets': 20000,

In [94]:
new_df = pd.DataFrame.from_dict(data_acc,orient ='index')
new_df = new_df.reset_index(drop=True)
new_df

,WordleID,Total_Tweets,Total_NonTrunc_Tweets,Date,Total_Extracted_Tweets,C1,P1,C2,P2,C3,...,C7,P7,Total_Hard_Tweets,PH1,PH2,PH3,PH4,PH5,PH6,PH7
0,254,20000,19727,2022-02-28,19556,374,1.91,1895,9.69,5893,...,152,0.78,589,0.68,10.36,35.48,39.22,11.21,2.72,0.34
1,255,20000,19501,2022-03-01,19429,200,1.03,599,3.08,3460,...,398,2.05,657,0.76,2.13,22.68,38.36,27.85,7.31,0.91
2,256,20000,19703,2022-03-02,19641,345,1.76,1812,9.23,5285,...,408,2.08,591,2.20,11.51,31.81,25.89,17.77,8.97,1.86
3,257,20000,19707,2022-03-03,19680,198,1.01,1737,8.83,5764,...,203,1.03,678,0.15,9.29,37.32,32.60,15.49,4.87,0.29
4,258,20000,19658,2022-03-04,19609,240,1.22,978,4.99,3847,...,328,1.67,745,0.27,3.62,23.62,42.42,22.42,6.71,0.94
5,259,20000,19723,2022-03-05,19681,194,0.99,2200,11.18,5243,...,450,2.29,723,0.41,14.25,28.08,26.69,19.92,8.30,2.35
6,260,20000,19750,2022-03-06,19716,169,0.86,1790,9.08,6401,...,195,0.99,724,0.55,9.12,43.92,30.11,11.88,3.31,1.10
7,261,20000,19731,2022-03-07,19689,194,0.99,2039,10.36,6032,...,175,0.89,660,1.21,10.00,36.52,34.70,13.48,4.09,NaN
8,262,20000,19687,2022-03-08,19655,249,1.27,1140,5.80,3777,...,430,2.19,709,0.14,5.36,18.19,34.13,25.11,14.95,2.12
9,263,20000,19709,2022-03-09,19673,220,1.12,1194,6.07,5198,...,223,1.13,744,0.67,6.18,30.65,36.96,19.89,4.70,0.94


In [95]:
if not os.path.exists("updated"):
    os.mkdir("updated")

### Removing NaN Values

In [96]:
new_df.isnull().sum().sum()

1

In [97]:
new_df = new_df.fillna('')
new_df

,WordleID,Total_Tweets,Total_NonTrunc_Tweets,Date,Total_Extracted_Tweets,C1,P1,C2,P2,C3,...,C7,P7,Total_Hard_Tweets,PH1,PH2,PH3,PH4,PH5,PH6,PH7
0,254,20000,19727,2022-02-28,19556,374,1.91,1895,9.69,5893,...,152,0.78,589,0.68,10.36,35.48,39.22,11.21,2.72,0.34
1,255,20000,19501,2022-03-01,19429,200,1.03,599,3.08,3460,...,398,2.05,657,0.76,2.13,22.68,38.36,27.85,7.31,0.91
2,256,20000,19703,2022-03-02,19641,345,1.76,1812,9.23,5285,...,408,2.08,591,2.20,11.51,31.81,25.89,17.77,8.97,1.86
3,257,20000,19707,2022-03-03,19680,198,1.01,1737,8.83,5764,...,203,1.03,678,0.15,9.29,37.32,32.60,15.49,4.87,0.29
4,258,20000,19658,2022-03-04,19609,240,1.22,978,4.99,3847,...,328,1.67,745,0.27,3.62,23.62,42.42,22.42,6.71,0.94
5,259,20000,19723,2022-03-05,19681,194,0.99,2200,11.18,5243,...,450,2.29,723,0.41,14.25,28.08,26.69,19.92,8.30,2.35
6,260,20000,19750,2022-03-06,19716,169,0.86,1790,9.08,6401,...,195,0.99,724,0.55,9.12,43.92,30.11,11.88,3.31,1.1
7,261,20000,19731,2022-03-07,19689,194,0.99,2039,10.36,6032,...,175,0.89,660,1.21,10.00,36.52,34.70,13.48,4.09,
8,262,20000,19687,2022-03-08,19655,249,1.27,1140,5.80,3777,...,430,2.19,709,0.14,5.36,18.19,34.13,25.11,14.95,2.12
9,263,20000,19709,2022-03-09,19673,220,1.12,1194,6.07,5198,...,223,1.13,744,0.67,6.18,30.65,36.96,19.89,4.70,0.94


In [98]:
new_df.isnull().sum().sum()

0

In [99]:
last_df = new_df.iloc[-1]
last_df
last_wid_df = pd.DataFrame(columns = ['Attempt','Count','Percentage','HardPercentage','Total_Tweets','Total_NonTrunc_Tweets','Total_Extracted_Tweets','Total_Hard_Tweets','iPhone','Android','Other'])
last_wid_df.at[1,'Attempt'] = 1

for i in range(1,8):
    last_wid_df.at[i,'Attempt'] = f"{i}/6"
    last_wid_df.at[i,'Count'] = getattr(last_df,f'C{i}')
    last_wid_df.at[i,'Percentage'] = (getattr(last_df,f'P{i}')/100).round(3)
    last_wid_df.at[i,'HardPercentage'] = getattr(last_df,f'PH{i}')
last_wid_df.at[7,'Attempt'] = 'x/6'
last_wid_df.at[1,'Total_Tweets'] = int(last_df.Total_Tweets)
last_wid_df.at[1,'Total_NonTrunc_Tweets'] = int(last_df.Total_NonTrunc_Tweets)
last_wid_df.at[1,'Total_Extracted_Tweets'] = int(last_df.Total_Extracted_Tweets)
last_wid_df.at[1,'Total_Hard_Tweets'] = int(last_df.Total_Hard_Tweets)
last_wid_df

,Attempt,Count,Percentage,HardPercentage,Total_Tweets,Total_NonTrunc_Tweets,Total_Extracted_Tweets,Total_Hard_Tweets,iPhone,Android,Other
1,1/6,386,0.02,1.13,20000,19290,19065,888,NaN,NaN,NaN
2,2/6,1287,0.068,4.39,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3/6,2860,0.15,13.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4/6,3321,0.174,15.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5/6,2953,0.155,14.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6/6,4248,0.223,23.09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,x/6,4010,0.21,28.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [100]:
source_df = df.loc[df['WordleID'] == df.iloc[-1].WordleID]
s = source_df.Source
c = s.value_counts()
p = s.value_counts(normalize=True).round(3).astype(float)
x = pd.DataFrame({'counts': c, 'per': p})
print(x[:2])
other_per  = p[2:].sum().round(3)
print(other_per)
last_wid_df.at[1,'iPhone'] = x.per[0]
last_wid_df.at[1,'Android'] = x.per[1]
last_wid_df.at[1,'Other'] = other_per
last_wid_df

                     counts    per
Twitter for iPhone    11669  0.605
Twitter for Android    4297  0.223
0.169


,Attempt,Count,Percentage,HardPercentage,Total_Tweets,Total_NonTrunc_Tweets,Total_Extracted_Tweets,Total_Hard_Tweets,iPhone,Android,Other
1,1/6,386,0.02,1.13,20000,19290,19065,888,0.605,0.223,0.169
2,2/6,1287,0.068,4.39,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3/6,2860,0.15,13.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4/6,3321,0.174,15.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5/6,2953,0.155,14.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6/6,4248,0.223,23.09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,x/6,4010,0.21,28.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
last_wid_df = last_wid_df.fillna('')
last_wid_df

,Attempt,Count,Percentage,HardPercentage,Total_Tweets,Total_NonTrunc_Tweets,Total_Extracted_Tweets,Total_Hard_Tweets,iPhone,Android,Other
1,1/6,386,0.020,1.13,20000,19290,19065,888,0.605,0.223,0.169
2,2/6,1287,0.068,4.39,,,,,,,
3,3/6,2860,0.150,13.40,,,,,,,
4,4/6,3321,0.174,15.32,,,,,,,
5,5/6,2953,0.155,14.64,,,,,,,
6,6/6,4248,0.223,23.09,,,,,,,
7,x/6,4010,0.210,28.04,,,,,,,


In [102]:
# df.groupby(["Source"])["Source"].count().nlargest(10)

In [103]:
# x = df.groupby(["Source"])["Source"].count().rename("Percentage").transform(lambda x: (x*100/x.sum()).round(1)).nlargest(10)
# # x['Per'] = 100 * x['Source']  / x['Source'].sum()
# x = x.reset_index()

# x

In [104]:
# s = df.Location
# c = s.value_counts()
# p = s.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
# x = pd.DataFrame({'counts': c, 'per': p})
# x

In [105]:
!pip3 install gspread
!pip3 install --upgrade google-api-python-client oauth2client

In [106]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials

In [107]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
g_auth_provider_x509_cert_url = user_secrets.get_secret("g_auth_provider_x509_cert_url")
g_auth_uri = user_secrets.get_secret("g_auth_uri")
g_client_email = user_secrets.get_secret("g_client_email")
g_client_id = user_secrets.get_secret("g_client_id")
g_client_x509_cert_url = user_secrets.get_secret("g_client_x509_cert_url")
g_private_key = user_secrets.get_secret("g_private_key")
g_private_key_id = user_secrets.get_secret("g_private_key_id")
g_projectid = user_secrets.get_secret("g_projectid")
g_token_uri = user_secrets.get_secret("g_token_uri")
g_type = user_secrets.get_secret("g_type")

In [108]:
credentials = {
      "type": g_type,
      "project_id": g_projectid,
      "private_key_id": g_private_key_id,
      "private_key": g_private_key.replace('\\n', '\n'),
      "client_email": g_client_email,
      "client_id": g_client_id,
      "auth_uri": g_auth_uri,
      "token_uri": g_token_uri,
      "auth_provider_x509_cert_url": g_auth_provider_x509_cert_url,
      "client_x509_cert_url": g_client_x509_cert_url
}

In [109]:
gc = gspread.service_account_from_dict(credentials)

In [110]:
sh = gc.open("WordleDataSheet")

print(sh.sheet1.get('A1'))

[['WordleID']]


In [111]:
sheet1 = sh.get_worksheet(0)
sheet1.clear()


{'spreadsheetId': '1IUZZwhFY9Esqv5X0JNvyqARHemPLBLUKWja4v8QMp4g',
 'clearedRange': 'Sheet1!A1:AA1000'}

In [112]:
sheet2 = sh.get_worksheet(1)
sheet2.clear()

{'spreadsheetId': '1IUZZwhFY9Esqv5X0JNvyqARHemPLBLUKWja4v8QMp4g',
 'clearedRange': 'Sheet2!A1:AA1000'}

In [113]:
sheet1.update([new_df.columns.values.tolist()] + new_df.values.tolist())


{'spreadsheetId': '1IUZZwhFY9Esqv5X0JNvyqARHemPLBLUKWja4v8QMp4g',
 'updatedRange': 'Sheet1!A1:AA13',
 'updatedRows': 13,
 'updatedColumns': 27,
 'updatedCells': 351}

In [114]:
sheet2.update([last_wid_df.columns.values.tolist()] + last_wid_df.values.tolist())

{'spreadsheetId': '1IUZZwhFY9Esqv5X0JNvyqARHemPLBLUKWja4v8QMp4g',
 'updatedRange': 'Sheet2!A1:K8',
 'updatedRows': 8,
 'updatedColumns': 11,
 'updatedCells': 88}